# Basic `alphatools` workflow

1. load data and meta data
2. basic minimal preprocessing
3. perform PCA
4. plot PCA embeddings, variance and loadings

In [ ]:
%load_ext autoreload
%autoreload 2

import logging
from anndata import AnnData
import numpy as np
import pandas as pd

import alphatools as at
from alphatools.pl.figure import create_figure, label_axes
from alphatools.pl.plots import Plots
from alphatools.pp.embeddings import pca
from alphatools.pp.metadata import add_core_proteome_mask

logging.basicConfig(level=logging.INFO)

## Load the data 

Dataset taken from datashare folder: https://datashare.biochem.mpg.de/s/sSYkOj22kM5AJ4O

In [ ]:
#### Chagne after merging, and use Lucas' code to load the DIA-NN data

# load protein groups into an AnnData object with index & columns as obs & var
url_data = "https://datashare.biochem.mpg.de/s/sSYkOj22kM5AJ4O/download?path=%2FSearch%20results%2FDIA-NN%201.9.2&files=report.pg_matrix.tsv"
df = pd.read_csv(url_data, sep="\t")

url_metadata = "https://datashare.biochem.mpg.de/s/sSYkOj22kM5AJ4O/download?path=%2F&files=simple_metadata.csv"
md = pd.read_csv(url_metadata, sep=",")

adata = AnnData(
    X=df.drop(columns=["Protein.Group", "Protein.Names", "Genes", "First.Protein.Description"]).to_numpy().T,
    obs=md,
    var=df[["Protein.Group", "Protein.Names", "Genes", "First.Protein.Description"]].set_index("Protein.Group"),
)

# overview of the data object
print(adata)

## Basic EDA on a synthetic example dataset:

1. Generate example data
2. Filter for data completeness on sample level
3. Visualize samples as histograms
4. Save data

### Filter by data completeness:

Remove features which have more than the allowed fraction of missing values

In [ ]:
print("The numeric data in the anndata object:")
display(adata.to_df().head())

print("The sample-level metadata in the anndata object:")
display(adata.obs.head())

print("The feature-level metadata in the anndata object:")
display(adata.var.head())

#  filter out features with more than 25 % missing values
adata = at.pp.filter_data_completeness(adata=adata, max_missing=0.25)

print("The numeric data in the anndata object:")
display(adata.to_df().head())

print("The sample-level metadata in the anndata object:")
display(adata.obs.head())

print("The feature-level metadata in the anndata object:")
display(adata.var.head())

## Creating new layers prior to preprocessing

This way, we can save the raw data and try different pp steps on the raw data.

In [ ]:
# we assume that the data is already log2 transformed
adata.layers["raw"] = adata.X.copy()

# log2 transform the data
adata.X = np.log2(adata.X + 1)

### Visualize the distribution of values in different levels of an observational metadata variable

In this example, check the distribution of "gene_1" expression values per cell type.

In [ ]:
# Apply the AxisManager to make axes iterable and apply consistent AlphaTools styling.
# Axes can also be accessed directly by indexing the axm object.
fig, axm = create_figure(nrows=1, ncols=2, figsize=(5, 3))

# Plot.histogram handles adata natively. Columns from the data and metadata are accessible
# Focus on the distribution of protein A1L0T0
ax = axm.next()
Plots.histogram(
    data=adata,
    value_column="A1L0T0",
    bins=20,
    legend="auto",
    ax=ax,
    hist_kwargs={"alpha": 0.5, "histtype": "stepfilled", "linewidth": 0.5, "edgecolor": "black"},
)
label_axes(ax, "A1L0T0", "Frequency", "Distribution of A1L0T0")

# Focus on the distribution of protein A1L0T0 in the different replicates
ax = axm.next()
Plots.histogram(
    data=adata,
    value_column="A1L0T0",
    color_column="replicate",
    bins=20,
    legend="auto",
    ax=ax,
    hist_kwargs={"alpha": 0.5, "histtype": "stepfilled", "linewidth": 0.5, "edgecolor": "black"},
)
label_axes(ax, "A1L0T0", "Frequency", "Distribution of A1L0T0 by replicate")

# # save figure
# save_figure(
#     fig=fig,
#     filename="sample_histogram.png",
#     output_dir=output_directory,
#     dpi=300,
#     transparent=False,
# )

### Running PCA

PCA can not be computed on matrices with missing values. Therefore, prior to PCA, we will create a list of 'core proteins' of proteins detected in all observations, save it in the feature meta data frame (adata.var) and run PCA on the shortlisted proteins. 

In [ ]:
# add a new column to the adata.var object with the name "isCore" to indicate whether the feature is part of the core proteome
# The core proteome is defined as the set of features that are detected in of the samples
add_core_proteome_mask(adata, layer="raw", new_column_name="is_core")

# view hoe many features are part of the core proteome
print("The number of features in the core proteome:")
print(adata.var["is_core"].value_counts())

# run PCA on the data
# this function is now implemented on sample level (PCA of the observations).
pca(adata, meta_data_mask_column_name="is_core", n_comps=10)

# view the PCA results
print("The dimensions of PC coordinates in the adata.obs are (n_obs x n_comp):")
print(adata.obsm["X_pca"].shape)
print("The PCA loadings in the adata.varm are (n_var x n_comp):")
print(adata.varm["PCs"].shape)
print("Ratio of explained variance (n_comp):")
print(adata.uns["pca"]["variance_ratio"])
print("The explained variance (n_comp):")
print(adata.uns["pca"]["variance"])

### Plot the PCA results 

In [ ]:
fig, axm = create_figure(1, 2, figsize=(8, 4))
ax = axm.next()

Plots.plot_pca(
    data=adata,
    ax=ax,
    pc_x=1,
    pc_y=2,
    label=False,
    label_column=None,
    pca_embeddings_layer_name="X_pca",
    pca_variance_layer_name="pca",
)

ax = axm.next()
# color the points by replicate
Plots.plot_pca(
    data=adata,
    ax=ax,
    pc_x=1,
    pc_y=2,
    label=False,
    label_column=None,
    color_column="replicate",
    pca_embeddings_layer_name="X_pca",
    pca_variance_layer_name="pca",
)

In [ ]:
fig, axm = create_figure(1, 1, figsize=(4, 4))
ax = axm.next()
Plots.scree_plot(data=adata, ax=ax, n_pcs=50, pca_variance_layer_name="pca")

In [ ]:
fig, axm = create_figure(1, 1, figsize=(4, 4))
ax = axm.next()
Plots.plot_pca_loadings(
    data=adata,
    ax=ax,
    dim=1,
    nfeatures=20,
)

In [ ]:
# Plot the PCA loadings for 2 PCs in 2D (this helps in identifying the features that contribute most to the PC plot)
fig, axm = create_figure(1, 1, figsize=(6, 6))
ax = axm.next()

Plots.plot_pca_loadings_2d(
    data=adata,
    loadings_name="PCs",
    ax=ax,
    pc_x=1,
    pc_y=2,
    nfeatures=20,
    add_labels=True,
    add_lines=True,
    scatter_kwargs=None,
)